<a href="https://colab.research.google.com/github/hertie-data-science-lab/tutorial-new-group-2-1/blob/main/tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tutorial: Transfer Learning for Flood Mapping Using Sentinel-1 Radar Imagery


# GRAD-E1394 Deep Learning - Assignment 3

Authors:


*   Aditi Joshi
*   Elena Murray
*   Leticia Figueiredo Collado
*   Sattiki Ganguly
*   Xiaohan Wu







# Memo - add more context

Flooding is one of the most damaging climate-related hazards today. It destroy infrastructure, reduce agricultural output, and threaten lives. Governments increasingly rely on satellite imagery to monitor floods, especially in regions lacking ground sensors. Sentinel-1, a freely available radar satellite, is particularly valuable because it can capture images regardless of cloud cover, exactly when floods occur.

However, interpreting satellite radar data requires technical expertise and manual processing. Deep learning models can help analysts produce rapid, large-scale flood maps, supporting emergy response, disaster insurance, infrastructure planning, and climate adaptation strategies.

This tutorial introduces transfer learning, a method that reuses knowledge learned by large pretrained CNNs and adapts them to a new task with limited labeled data. This approach is widely used in governments, including FEMA (USA), the European Copernicus Program, and India's National Remote Sensing Center.

This tutorial provides a practical and reproducible workflow that supports flood-risk monitoring, climate-resilience planning, and rapid diasaster assessment. It equips analysts with the foundational skills needed to support data-driven policy in environmental management and emergency decision-making.

## 1. Overview

to be added

## 2. Background & Prerequisites

### Conceptual background

* **Convolutional Neural Networks (CNNs):**  
  Neural networks that apply learnable filters across images to detect patterns like edges, textures, and shapes.

* **Transfer Learning:**  
  Instead of training a CNN from scratch, we:
  1. Start from a model pretrained on a large dataset (e.g. ImageNet with millions of natural images).
  2. Replace and fine-tune only the last layers for our task.
  3. Benefit from previously learned low-level features, saving data and compute.

* **Sentinel-1 Radar (SAR):**  
  A satellite that sends microwave pulses and measures backscatter. Flooded areas often appear darker because smooth water surfaces reflect energy away from the satellite.


## 3. Software Requirements

This notebook is designed for **Google Colab**, but it also works in a local Jupyter environment if you have access to the data.

We use:

* `torch`, `torchvision` – deep learning
* `numpy`, `pandas` – data handling
* `rasterio` – reading GeoTIFF satellite images
* `matplotlib` – plotting
* `sklearn` – splitting data, metrics

Run the following cell once to install dependencies.

In [12]:
!pip install rasterio
!pip install torch torchvision torchaudio
!pip install scikit-learn

In [13]:
import os
import glob
import random
from pathlib import Path

import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision.models import resnet18
from torchvision import transforms

from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split

import rasterio

# For reproducibility
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

Using device: cpu


## 4. Data Description – Sen1Floods11 (Small Subset)

We work with a **small sample** of the [Sen1Floods11](http://openaccess.thecvf.com/content_CVPRW_2020/html/w11/Bonafilia_Sen1Floods11_A_Georeferenced_Dataset_to_Train_and_Test_Deep_Learning_CVPRW_2020_paper.html) dataset.

Each *event* consists of:

* **S1**: Sentinel-1 radar image tile (512×512, bands: VV & VH).
* **QC**: Hand-labeled ground truth mask (512×512), values:
  * `1` = water (flooded)
  * `0` = non-water
  * `-1` = no data / invalid


In [ ]:
!pip install -q gcsfs

In [ ]:
!mkdir -p /content/data/floods11

In [ ]:
!gsutil -m rsync -r gs://sen1floods11 /content/data/floods11


# List top-level folders in the bucket
!gsutil ls gs://sen1floods11

# Look inside the data folder (names may vary slightly; adjust if needed)
!gsutil ls gs://sen1floods11/data/
!gsutil ls gs://sen1floods11/data/QC | head
!gsutil ls gs://sen1floods11/data/S1 | head

Streaming output truncated to the last 5000 lines.
Copying gs://sen1floods11/v1.1/catalog/sen1floods11_weak_labeled_label/Mekong_272771_label/Mekong_272771_label.json...
Copying gs://sen1floods11/v1.1/catalog/sen1floods11_weak_labeled_label/Mekong_2728_label/Mekong_2728_label.json...
Copying gs://sen1floods11/v1.1/catalog/sen1floods11_weak_labeled_label/Mekong_2740940_label/Mekong_2740940_label.json...
Copying gs://sen1floods11/v1.1/catalog/sen1floods11_weak_labeled_label/Mekong_2742505_label/Mekong_2742505_label.json...
Copying gs://sen1floods11/v1.1/catalog/sen1floods11_weak_labeled_label/Mekong_2746141_label/Mekong_2746141_label.json...
Copying gs://sen1floods11/v1.1/catalog/sen1floods11_weak_labeled_label/Mekong_2750903_label/Mekong_2750903_label.json...
Copying gs://sen1floods11/v1.1/catalog/sen1floods11_weak_labeled_label/Mekong_275529_label/Mekong_275529_label.json...
Copying gs://sen1floods11/v1.1/catalog/sen1floods11_weak_labeled_label/Mekong_2754109_label/Mekong_2754109_label

KeyboardInterrupt: 

In [ ]:
data_dir = Path('data/floods11')

# Check if dataset exists
if not data_dir.exists():
    print("⚠️ Dataset not found. Please run the download commands above.")
else:
    print("📁 Dataset Structure:")
    # Show first few levels
    subdirs = [d for d in data_dir.iterdir() if d.is_dir()]
    for subdir in subdirs[:5]:
        print(f"  📂 {subdir.name}")
        files = list(subdir.glob('*'))[:3]
        for f in files:
            print(f"    📄 {f.name}")

NameError: name 'Path' is not defined